# Monitor Deloitte Analytics Challenge - Step 1, Data cleanning and Variable creation. 

Xiaoyan Zhou

# Table of Content: 

* [1. Lag 1 week dataset](#1.-Lag 1 week dataset)
* [2. Price elasticity](#2.-Price elasticity)


In [9]:
# loading libraries
import pandas as pd
import numpy as np

In [10]:
#read data
product = pd.read_csv("products.csv")
sales = pd.read_csv("sales small.csv")
price_change = pd.read_csv("price changes.csv")

## 1.lag 1 week dataset

Using this week's data to predict next week's sales volume.

*Unchange features:

channel, country,group~class, product.season

*Last week features:( at the end of previous week)

all stocks related features, price change before

*This weeks's features:

Price change this week, season

*dependent variable:
next week's sales volume

*Method: joining the existed dataset with next week's sales volume and season.

Then generate the csv for four groups

In [11]:
#aim: target the price change with the correct product, week, channel,country.

#Step:join the sales dataset with price change dataset, left join, on four keys:
#Product ID, Week key (Year and week number),Channel (Online/Stores),Country ID


full = pd.merge(sales, price_change,how = "left", 
                left_on = ["sales_small.ProductID","sales_small.WeekKey","sales_small.Channel","sales_small.Country"],
                right_on =[ "price_changes.ProductID","price_changes.WeekKey","price_changes.Channel","price_changes.Country"])

### Variable creation 1: price change this week 

In [12]:
# calculate the amount of price change each week of each product
full['price_change_this_week'] = full["price_changes.previous_CSP"] - full["price_changes.CSP"]
# cumulated price change
full['total_price_change'] = full['price_changes.OSP'] - full["price_changes.CSP"]

In [13]:
# adding the product type to the full dataset
full = pd.merge(full, product, how = "left", left_on = "sales_small.ProductID", right_on = "products.ProductID")

## variable creation 2: season

In [14]:
#define a function to generate season base on weekkey

def season(weekkey):
    week = int(str(weekkey)[-2:])
    if week<=13:
        return("Spring")
    elif week<= 26:
        return("Summer")
    elif week<=39:
        return("Autumn")
    else:
        return("Winter")

In [15]:
full["season"] = full["sales_small.WeekKey"].apply(lambda x: season(x))

In [16]:
full_final = full.fillna(0)
full_final["price_change_before"] = full_final['total_price_change']- full_final['price_change_this_week']

In [17]:
def nextweek(weekkey):
    if weekkey==201552:
        return (201601)
    else:
        return(weekkey +1)

In [19]:
# merge data:
full_final_original = full_final
full_final.is_copy = False
full_final["next_week"] = full_final["sales_small.WeekKey"].apply(lambda x:nextweek(x))
full_final

full_final_whole = pd.merge(full_final, full_final, how = "left", 
                            left_on = ["sales_small.ProductID","next_week","sales_small.Channel","sales_small.Country"],
                right_on =["sales_small.ProductID","sales_small.WeekKey",
                           "sales_small.Channel","sales_small.Country"])


full_final_whole.head()

,sales_small.ProductID,sales_small.Channel,sales_small.Country,sales_small.WeekKey_x,sales_small.CSP_x,sales_small.SalesVolume_x,sales_small.TotalStockVolume_x,sales_small.StoreStockVolume_x,sales_small.DepotStockVolume_x,sales_small.FutureCommitmentVolume_x,...,total_price_change_y,products.ProductID_y,products.Season_y,products.Group_y,products.SubGroup_y,products.Class_y,products.SubClass_y,season_y,price_change_before_y,next_week_y
0,135fc45e,Stores,B,201502,18.984,3,39,6,33,0,...,0.0,135fc45e,L,26387251,c3567a18,8face522,641d430c,Spring,0.0,201504.0
1,1ff41410,Stores,A,201502,15.833,2,2,2,0,0,...,0.0,1ff41410,L,bca94c97,44b005af,c409cf28,8a0d6232,Spring,0.0,201504.0
2,22125db2,Stores,B,201502,18.084,1,6,2,3,0,...,0.0,22125db2,L,bca94c97,44b005af,c409cf28,8a0d6232,Spring,0.0,201504.0
3,25e07883,Stores,B,201502,13.583,6,6,2,4,0,...,0.0,25e07883,L,26387251,14edd834,96d57a3c,4e83142f,Spring,0.0,201504.0
4,2729a59e,Stores,B,201502,14.483,8,117,7,8,102,...,0.0,2729a59e,L,edf80f3a,edf80f3a,475fad98,76b05919,Spring,0.0,201504.0


In [20]:
#select dependent variable and independent variables
data = full_final_whole[['sales_small.ProductID',
 'sales_small.WeekKey_x',
 'sales_small.Channel',
 'sales_small.Country',
 'sales_small.CSP_x',
 'sales_small.SalesVolume_x',
 'sales_small.TotalStockVolume_x',
 'sales_small.StoreStockVolume_x',
 'sales_small.DepotStockVolume_x',
 'sales_small.FutureCommitmentVolume_x',
 'sales_small.IntakeVolume_x',
 'sales_small.StoresWithStockCount_x',
 'sales_small.StoresWithSalesCount_x',
 'price_change_this_week_x',
 'price_change_before_x', #price change before
 'products.Season_x',
 'products.Group_x',
 'products.SubGroup_x',
 'products.Class_x',
 'products.SubClass_x',
 'sales_small.SalesVolume_y',# to be predict
 'price_change_this_week_y',# next week price change
 'season_y']]# next week's season
                        

data.columns = ['sales_small.ProductID',
 'sales_small.WeekKey',
 'sales_small.Channel',
 'sales_small.Country',
 'sales_small.CSP',
 'sales_small.SalesVolume',
 'sales_small.TotalStockVolume',
 'sales_small.StoreStockVolume',
 'sales_small.DepotStockVolume',
 'sales_small.FutureCommitmentVolume',
 'sales_small.IntakeVolume',
 'sales_small.StoresWithStockCount',
 'sales_small.StoresWithSalesCount',
 'price_change_tw',
 'price_change_before', #price change before
 'products.Season',
 'products.Group',
 'products.SubGroup',
 'products.Class',
 'products.SubClass',
 'sales_small.SalesVolume_nw',# to be predict
 'price_change_nw',# next week price change
 'season_nw']

data.head()

,sales_small.ProductID,sales_small.WeekKey,sales_small.Channel,sales_small.Country,sales_small.CSP,sales_small.SalesVolume,sales_small.TotalStockVolume,sales_small.StoreStockVolume,sales_small.DepotStockVolume,sales_small.FutureCommitmentVolume,...,price_change_tw,price_change_before,products.Season,products.Group,products.SubGroup,products.Class,products.SubClass,sales_small.SalesVolume_nw,price_change_nw,season_nw
0,135fc45e,201502,Stores,B,18.984,3,39,6,33,0,...,0.0,0.0,L,26387251,c3567a18,8face522,641d430c,3.0,0.0,Spring
1,1ff41410,201502,Stores,A,15.833,2,2,2,0,0,...,0.0,0.0,L,bca94c97,44b005af,c409cf28,8a0d6232,4.0,0.0,Spring
2,22125db2,201502,Stores,B,18.084,1,6,2,3,0,...,0.0,0.0,L,bca94c97,44b005af,c409cf28,8a0d6232,1.0,0.0,Spring
3,25e07883,201502,Stores,B,13.583,6,6,2,4,0,...,2.7,0.0,L,26387251,14edd834,96d57a3c,4e83142f,17.0,0.0,Spring
4,2729a59e,201502,Stores,B,14.483,8,117,7,8,102,...,0.9,0.0,L,edf80f3a,edf80f3a,475fad98,76b05919,10.0,0.0,Spring


In [ ]:
# for 4 groups of products, form 4 csv file 
D26387251 = data.loc[data['products.Group'] == '26387251']
D26387251.to_csv("D26387251.csv")

Dbca94c97 = data.loc[data['products.Group'] == 'bca94c97']
Dbca94c97.to_csv("Dbca94c97.csv")

Dedf80f3a = data.loc[data['products.Group'] == 'edf80f3a']
Dedf80f3a.to_csv("Dedf80f3a.csv")

D606565a1 = data.loc[data['products.Group'] == '606565a1']
D606565a1.to_csv("D606565a1.csv")

## 2.Price elasticity

In [33]:
full_elastic = full

In [34]:
def previous_week(weekkey):
    if weekkey==201601:
        return (201552)
    else:
        return(weekkey -1)

In [35]:
full_elastic.is_copy = False
full_elastic["Previous_week"] = full_elastic["sales_small.WeekKey"].apply(lambda x:previous_week(x))
full_elastic.head()

,sales_small.ProductID,sales_small.Channel,sales_small.Country,sales_small.WeekKey,sales_small.CSP,sales_small.SalesVolume,sales_small.TotalStockVolume,sales_small.StoreStockVolume,sales_small.DepotStockVolume,sales_small.FutureCommitmentVolume,...,price_change_this_week,total_price_change,products.ProductID,products.Season,products.Group,products.SubGroup,products.Class,products.SubClass,season,Previous_week
0,135fc45e,Stores,B,201502,18.984,3,39,6,33,0,...,NaN,NaN,135fc45e,L,26387251,c3567a18,8face522,641d430c,Spring,201501
1,1ff41410,Stores,A,201502,15.833,2,2,2,0,0,...,NaN,NaN,1ff41410,L,bca94c97,44b005af,c409cf28,8a0d6232,Spring,201501
2,22125db2,Stores,B,201502,18.084,1,6,2,3,0,...,NaN,NaN,22125db2,L,bca94c97,44b005af,c409cf28,8a0d6232,Spring,201501
3,25e07883,Stores,B,201502,13.583,6,6,2,4,0,...,2.7,2.7,25e07883,L,26387251,14edd834,96d57a3c,4e83142f,Spring,201501
4,2729a59e,Stores,B,201502,14.483,8,117,7,8,102,...,0.9,0.9,2729a59e,L,edf80f3a,edf80f3a,475fad98,76b05919,Spring,201501


In [36]:
# self merge to get the previous weeks sales volumes and price
full_elastic_whole = pd.merge(full_elastic, full_elastic, how = "left", 
                              left_on = ["sales_small.ProductID","Previous_week","sales_small.Channel","sales_small.Country"],
                right_on =["sales_small.ProductID","sales_small.WeekKey","sales_small.Channel","sales_small.Country"])


list(full_elastic_whole)
full_elastic_whole  = full_elastic_whole[['sales_small.ProductID',
 'sales_small.Channel',
 'sales_small.Country',
 'sales_small.WeekKey_x',
 'sales_small.CSP_x',
 'sales_small.SalesVolume_x',
 'price_change_this_week_x',
 'sales_small.WeekKey_y',
 'sales_small.SalesVolume_y',
 'sales_small.CSP_y']]
full_elastic_whole = full_elastic_whole.dropna()
full_elastic_whole.head()

,sales_small.ProductID,sales_small.Channel,sales_small.Country,sales_small.WeekKey_x,sales_small.CSP_x,sales_small.SalesVolume_x,price_change_this_week_x,sales_small.WeekKey_y,sales_small.SalesVolume_y,sales_small.CSP_y
950,135fc45e,Stores,B,201507,16.283,13,2.701,201506.0,4.0,18.984
1000,3b325781,Online,B,201507,16.733,5,4.501,201506.0,3.0,21.234
1019,429309b,Stores,A,201507,13.493,2,2.340,201506.0,0.0,15.833
1020,431eec16,Online,B,201507,14.483,3,3.601,201506.0,2.0,18.084
1022,44d4fa38,Stores,B,201507,12.232,2,1.351,201506.0,0.0,13.583


In [37]:
full_elastic_whole.shape

(2640, 10)

In [38]:
# calculate volume change percentage, price change percentage
full_elastic_whole["volume_change_percentage"] = (full_elastic_whole["sales_small.SalesVolume_x"]-
                                                  full_elastic_whole["sales_small.SalesVolume_y"])/full_elastic_whole["sales_small.SalesVolume_y"]
full_elastic_whole["price_change_percentage"] = (full_elastic_whole["sales_small.CSP_x"] -
                                                 full_elastic_whole["sales_small.CSP_y"])/full_elastic_whole["sales_small.CSP_y"]


# replace volume change NAN with 0,drop inf and -inf to get a conservative estimate of price elasticity
full_elastic_whole.is_copy = False
full_elastic_whole = full_elastic_whole[~full_elastic_whole.isin([np.inf, -np.inf]).any(1)]
full_elastic_whole["volume_change_percentage"] = full_elastic_whole["volume_change_percentage"].replace(np.nan, 0)
full_elastic_whole.head()

,sales_small.ProductID,sales_small.Channel,sales_small.Country,sales_small.WeekKey_x,sales_small.CSP_x,sales_small.SalesVolume_x,price_change_this_week_x,sales_small.WeekKey_y,sales_small.SalesVolume_y,sales_small.CSP_y,volume_change_percentage,price_change_percentage
950,135fc45e,Stores,B,201507,16.283,13,2.701,201506.0,4.0,18.984,2.250000,-0.142278
1000,3b325781,Online,B,201507,16.733,5,4.501,201506.0,3.0,21.234,0.666667,-0.211971
1020,431eec16,Online,B,201507,14.483,3,3.601,201506.0,2.0,18.084,0.500000,-0.199126
1087,7c3602f1,Online,B,201507,14.033,8,2.250,201506.0,3.0,16.283,1.666667,-0.138181
1088,7c3602f1,Stores,A,201507,15.190,35,2.894,201506.0,13.0,18.084,1.692308,-0.160031


In [39]:
# calculate elasticity

full_elastic_whole.is_copy = False
full_elastic_whole["elasticity"] = full_elastic_whole["volume_change_percentage"]/full_elastic_whole["price_change_percentage"]
full_elasticity = full_elastic_whole[['sales_small.ProductID','elasticity']]
a = full_elasticity.groupby(['sales_small.ProductID']).mean()

In [40]:
a.reset_index(inplace = True)

In [41]:
a = pd.merge(a,product, how = "left", left_on = "sales_small.ProductID", right_on = "products.ProductID")
a.head()

,sales_small.ProductID,elasticity,products.ProductID,products.Season,products.Group,products.SubGroup,products.Class,products.SubClass
0,10516724,-12.215056,10516724,L,bca94c97,c830b21c,6ad22ddd,2f6d2d03
1,1145adc2,-6.333153,1145adc2,L,bca94c97,44b005af,bb900370,c6ea2cc6
2,11f52f37,-25.665447,11f52f37,L,26387251,14edd834,96d57a3c,3906b741
3,122cd18,-5.697407,122cd18,L,26387251,14edd834,96d57a3c,3906b741
4,126d41d3,-33.470316,126d41d3,L,c8e7b2df,c8e7b2df,6e175527,aa81453f


In [ ]:
a.to_csv("product_elasticity.csv")